<a href="https://colab.research.google.com/github/mtavakoliangit/ena/blob/main/ena_1device_garage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive 
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
import pandas as pd
from datetime import datetime, date

In [3]:
#load data from gdrive
df_1dev_garage = pd.read_csv('gdrive/My Drive/ENA_db/garage_onedevice.csv')

In [4]:
df_1dev_garage.head()

,_id,day,device_id,esp_version,first,last,nsamples,reading_heat_time_q_max,reading_heat_time_q_min,reading_temperature_U6,samples,st_version
0,5faf735c9cf7b6988ae0e760,2020-11-13,09963013926712196740163043482,"[""0.0.45""]",23:04:24,23:59:21,43,39628,39628,8.0,"[{""time_stamp"":""23:04:24"",""setpoint_temperatur...","[""000.000.032""]"
1,5faf8079a183f18be8354312,2020-11-14,09963013926712196740163043482,"[""0.0.45""]",00:00:08,23:54:40,1331,46648,39628,8.5,"[{""time_stamp"":""00:00:08"",""setpoint_temperatur...","[""000.000.032""]"
2,5fb0d21da183f18be836844e,2020-11-15,09963013926712196740163043482,"[""0.0.45""]",00:00:44,23:59:39,1335,47309,46648,7.0,"[{""time_stamp"":""00:00:44"",""setpoint_temperatur...","[""000.000.032""]"
3,5fb22353a183f18be837d3db,2020-11-16,09963013926712196740163043482,"[""0.0.45""]",00:00:34,23:58:56,1332,50220,47309,8.0,"[{""time_stamp"":""00:00:44"",""setpoint_temperatur...","[""000.000.032""]"
4,5fb374f244ad208acea906b4,2020-11-17,09963013926712196740163043482,"[""0.0.45""]",00:00:00,23:59:31,1332,51868,50220,9.0,"[{""time_stamp"":""00:00:00"",""setpoint_temperatur...","[""000.000.032""]"


In [5]:
df_1dev_garage.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 367 entries, 0 to 366
Data columns (total 12 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   _id                      367 non-null    object 
 1   day                      367 non-null    object 
 2   device_id                367 non-null    object 
 3   esp_version              367 non-null    object 
 4   first                    367 non-null    object 
 5   last                     367 non-null    object 
 6   nsamples                 367 non-null    int64  
 7   reading_heat_time_q_max  367 non-null    int64  
 8   reading_heat_time_q_min  367 non-null    int64  
 9   reading_temperature_U6   367 non-null    float64
 10  samples                  367 non-null    object 
 11  st_version               367 non-null    object 
dtypes: float64(1), int64(3), object(8)
memory usage: 34.5+ KB


In [6]:
df_1dev_garage['device_id'].unique()

array(['09963013926712196740163043482'], dtype=object)

In [7]:
#filter data for a specific device
df_device = df_1dev_garage[df_1dev_garage['device_id'] == '09963013926712196740163043482']

In [8]:
df_device.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 367 entries, 0 to 366
Data columns (total 12 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   _id                      367 non-null    object 
 1   day                      367 non-null    object 
 2   device_id                367 non-null    object 
 3   esp_version              367 non-null    object 
 4   first                    367 non-null    object 
 5   last                     367 non-null    object 
 6   nsamples                 367 non-null    int64  
 7   reading_heat_time_q_max  367 non-null    int64  
 8   reading_heat_time_q_min  367 non-null    int64  
 9   reading_temperature_U6   367 non-null    float64
 10  samples                  367 non-null    object 
 11  st_version               367 non-null    object 
dtypes: float64(1), int64(3), object(8)
memory usage: 37.3+ KB


In [9]:
df_device.describe()

,nsamples,reading_heat_time_q_max,reading_heat_time_q_min,reading_temperature_U6
count,367.000000,367.000000,367.000000,367.000000
mean,1183.136240,252665.504087,248741.607629,9.787466
std,509.011802,216929.856153,216820.698200,5.652124
min,1.000000,0.000000,0.000000,-1.500000
25%,1331.500000,4730.000000,4730.000000,6.000000
50%,1431.000000,231995.000000,226432.000000,7.500000
75%,1448.000000,456718.500000,456417.000000,11.000000
max,1511.000000,562996.000000,562996.000000,26.500000


In [10]:
df_device.head()

,_id,day,device_id,esp_version,first,last,nsamples,reading_heat_time_q_max,reading_heat_time_q_min,reading_temperature_U6,samples,st_version
0,5faf735c9cf7b6988ae0e760,2020-11-13,09963013926712196740163043482,"[""0.0.45""]",23:04:24,23:59:21,43,39628,39628,8.0,"[{""time_stamp"":""23:04:24"",""setpoint_temperatur...","[""000.000.032""]"
1,5faf8079a183f18be8354312,2020-11-14,09963013926712196740163043482,"[""0.0.45""]",00:00:08,23:54:40,1331,46648,39628,8.5,"[{""time_stamp"":""00:00:08"",""setpoint_temperatur...","[""000.000.032""]"
2,5fb0d21da183f18be836844e,2020-11-15,09963013926712196740163043482,"[""0.0.45""]",00:00:44,23:59:39,1335,47309,46648,7.0,"[{""time_stamp"":""00:00:44"",""setpoint_temperatur...","[""000.000.032""]"
3,5fb22353a183f18be837d3db,2020-11-16,09963013926712196740163043482,"[""0.0.45""]",00:00:34,23:58:56,1332,50220,47309,8.0,"[{""time_stamp"":""00:00:44"",""setpoint_temperatur...","[""000.000.032""]"
4,5fb374f244ad208acea906b4,2020-11-17,09963013926712196740163043482,"[""0.0.45""]",00:00:00,23:59:31,1332,51868,50220,9.0,"[{""time_stamp"":""00:00:00"",""setpoint_temperatur...","[""000.000.032""]"


In [11]:
df_device = df_device[['day', 'nsamples', 'samples']]

In [12]:
df_device.head()

,day,nsamples,samples
0,2020-11-13,43,"[{""time_stamp"":""23:04:24"",""setpoint_temperatur..."
1,2020-11-14,1331,"[{""time_stamp"":""00:00:08"",""setpoint_temperatur..."
2,2020-11-15,1335,"[{""time_stamp"":""00:00:44"",""setpoint_temperatur..."
3,2020-11-16,1332,"[{""time_stamp"":""00:00:44"",""setpoint_temperatur..."
4,2020-11-17,1332,"[{""time_stamp"":""00:00:00"",""setpoint_temperatur..."


In [13]:
df_device['day']= pd.to_datetime(df_device['day'])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [14]:
data_comp = pd.DataFrame(columns=['DateTime', 'setpoint_temp', 'setpoint_fan', 'reading_temp'])

In [15]:
for i in range(0, df_device.shape[0]):
  print(i)
  data_string = df_device['samples'][i].split()
  data_Series_format = pd.Series(data_string)
  data = data_Series_format.str.split(r'"time_stamp":"|","setpoint_temperature":|,"setpoint_fan":|,"reading_temperature_U6":|},{|}]', expand=True)
  counter = 1
  while counter <= data.shape[1] - 1:
    data[counter] = pd.to_datetime(data[counter],format= '%H:%M:%S' ).dt.time
    data[counter+1] = float(data[counter+1])
    data[counter+2] = float(data[counter+2])
    data[counter+3] = float(data[counter+3])
    counter = counter + 5
  j = 1
  n = 1
  while n < df_device['nsamples'][i]:
    dt = df_device['day'][i].replace(hour=data[j][0].hour, minute=data[j][0].minute, second=data[j][0].second)
    settemp = data[j+1][0]
    setfan = data[j+2][0]
    temp = data[j+3][0]
    data_comp = data_comp.append({'DateTime': dt, 'setpoint_temp': settemp, 'setpoint_fan': setfan, 'reading_temp': temp}, ignore_index=True)
    n = n + 1
    j = j + 5

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [17]:
data_comp.to_csv(r'gdrive/My Drive/ENA_db/garage_onedevice_formatted.csv')